In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img
%matplotlib inline

In [ ]:
print(os.listdir("/kaggle/input"))



In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,10))
img_name='NORMAL2-IM-0523-0001.jpeg'
img_norm=load_img('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/NORMAL/' + img_name)
ax[0].imshow(img_norm)
ax[0].set_title('NORMAL')

img_name='person1343_virus_2317.jpeg'
img_norm=load_img('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train/PNEUMONIA/' + img_name)

ax[1].imshow(img_norm)
ax[0].set_title('PNEUMANIA')


In [ ]:
img_width,img_height=150,150
train_data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/train'
validation_data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/val'
test_data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray/test'
input_shape = (img_width, img_height, 3)


In [ ]:
model=Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(16, (1, 1),padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

In [ ]:
test_generator=test_datagen.flow_from_directory(test_data_dir,target_size=(img_width,img_height)
                                               ,batch_size=16,class_mode='binary')

In [ ]:
nb_train_samples = 5217
nb_validation_samples = 17

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // 16,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples //16)


In [ ]:
scores = model.evaluate_generator(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))